# 数据采集

In [1]:
#首先导入使用的包，使用request进行网页请求，使用pandas保存数据
import requests
import pandas as pd
import time 
pd.set_option('max_rows',500)
#设置请求头，伪装为浏览器
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
}
#发起请求，将找到的第一个数据源作为请求目标
url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-total'   # 定义要访问的地址
r = requests.get(url, headers=headers)  # 使用requests发起请求

print(r.status_code)  # 查看请求状态
print(type(r.text))
print(len(r.text))

200
<class 'str'>
375807


In [2]:
#可以看到返回后的内容是一个几十万长度的字符串，
#由于字符串格式不方便进行分析，并且在网页预览中发现数据为类似字典的json格式，
#所以我们将其转为json格式
import json
data_json = json.loads(r.text)
data_json.keys()

dict_keys(['reqId', 'code', 'msg', 'data', 'timestamp'])

In [3]:
data = data_json['data'] # 取出json中的数据
data.keys()

dict_keys(['chinaTotal', 'chinaDayList', 'lastUpdateTime', 'overseaLastUpdateTime', 'areaTree'])

# 实时数据爬取

# 全国各省实时数据爬取

In [4]:
data_province = data['areaTree'][2]['children']  # 取出中国各省的实时数据

type(data_province)

list

In [5]:
data_province[0].keys() # 查看每个省键名称

dict_keys(['today', 'total', 'extData', 'name', 'id', 'lastUpdateTime', 'children'])

In [6]:
for i in range(len(data_province)): # 遍历查看各省名称、更新时间
    print(data_province[i]['name'],data_province[i]['lastUpdateTime'])
    if i == 10:
        break

湖北 2021-01-07 08:22:59
香港 2021-01-07 17:13:41
广东 2021-01-07 08:37:57
上海 2021-01-07 15:48:25
浙江 2021-01-07 08:22:58
河南 2021-01-07 08:22:58
湖南 2021-01-07 08:22:59
北京 2021-01-07 08:22:58
安徽 2021-01-07 08:22:58
新疆 2021-01-07 08:22:58
黑龙江 2021-01-07 15:48:25


In [7]:
test_dict = [{'a':1,
              'b':2,
              'c':3,},
             
             {'a':111,
              'b':222}]
pd.DataFrame(test_dict)

,a,b,c
0,1,2,3.0
1,111,222,NaN


In [8]:
pd.DataFrame(data_province).head() # 直接生成数据效果并不理想

,today,total,extData,name,id,lastUpdateTime,children
0,"{'confirm': 0, 'suspect': None, 'heal': 0, 'de...","{'confirm': 68149, 'suspect': 0, 'heal': 63637...",{},湖北,420000,2021-01-07 08:22:59,"[{'today': {'confirm': 0, 'suspect': None, 'he..."
1,"{'confirm': 25, 'suspect': None, 'heal': 131, ...","{'confirm': 9107, 'suspect': 0, 'heal': 8258, ...",{},香港,810000,2021-01-07 17:13:41,"[{'today': {'confirm': 25, 'suspect': None, 'h..."
2,"{'confirm': 2, 'suspect': None, 'heal': 3, 'de...","{'confirm': 2062, 'suspect': 0, 'heal': 2018, ...",{},广东,440000,2021-01-07 08:37:57,"[{'today': {'confirm': 2, 'suspect': None, 'he..."
3,"{'confirm': 5, 'suspect': None, 'heal': 5, 'de...","{'confirm': 1543, 'suspect': 0, 'heal': 1442, ...",{},上海,310000,2021-01-07 15:48:25,"[{'today': {'confirm': 5, 'suspect': None, 'he..."
4,"{'confirm': 0, 'suspect': None, 'heal': 0, 'de...","{'confirm': 1306, 'suspect': 0, 'heal': 1293, ...",{},浙江,330000,2021-01-07 08:22:58,"[{'today': {'confirm': 0, 'suspect': None, 'he..."


In [9]:
# 获取id、lastUpdateTime、name
info = pd.DataFrame(data_province)[['id','lastUpdateTime','name']]
info.head()

,id,lastUpdateTime,name
0,420000,2021-01-07 08:22:59,湖北
1,810000,2021-01-07 17:13:41,香港
2,440000,2021-01-07 08:37:57,广东
3,310000,2021-01-07 15:48:25,上海
4,330000,2021-01-07 08:22:58,浙江


In [10]:
# 获取today中的数据
today_data = pd.DataFrame([province['today'] for province in data_province ]) 
today_data.head()

,confirm,suspect,heal,dead,severe,storeConfirm
0,0,None,0,0,None,0
1,25,None,131,1,None,-74
2,2,None,3,0,None,-1
3,5,None,5,0,None,0
4,0,None,0,0,None,0


In [11]:
['today_'+i for i in today_data.columns]

['today_confirm',
 'today_suspect',
 'today_heal',
 'today_dead',
 'today_severe',
 'today_storeConfirm']

In [12]:
today_data.columns = ['today_'+i for i in today_data.columns] # 由于today中键名和total键名相同，因此需要修改列名称

today_data.head()

,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm
0,0,None,0,0,None,0
1,25,None,131,1,None,-74
2,2,None,3,0,None,-1
3,5,None,5,0,None,0
4,0,None,0,0,None,0


In [13]:
# 获取total中的数据
total_data = pd.DataFrame([province['total'] for province in data_province ])
total_data.columns = ['total_'+i for i in total_data.columns]
total_data.head()

,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_newConfirm,total_newDead,total_newHeal
0,68149,0,63637,4512,0,0,NaN,NaN,NaN
1,9107,0,8258,154,0,0,NaN,NaN,NaN
2,2062,0,2018,8,0,0,NaN,NaN,NaN
3,1543,0,1442,7,0,0,NaN,NaN,NaN
4,1306,0,1293,1,0,0,NaN,NaN,NaN


In [14]:
pd.concat([info,total_data,today_data],axis=1).head() # 将三个数据合并

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_newConfirm,total_newDead,total_newHeal,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm
0,420000,2021-01-07 08:22:59,湖北,68149,0,63637,4512,0,0,NaN,NaN,NaN,0,None,0,0,None,0
1,810000,2021-01-07 17:13:41,香港,9107,0,8258,154,0,0,NaN,NaN,NaN,25,None,131,1,None,-74
2,440000,2021-01-07 08:37:57,广东,2062,0,2018,8,0,0,NaN,NaN,NaN,2,None,3,0,None,-1
3,310000,2021-01-07 15:48:25,上海,1543,0,1442,7,0,0,NaN,NaN,NaN,5,None,5,0,None,0
4,330000,2021-01-07 08:22:58,浙江,1306,0,1293,1,0,0,NaN,NaN,NaN,0,None,0,0,None,0


In [15]:
# 将提取数据的方法封装为函数
def get_data(data,info_list):
    info = pd.DataFrame(data)[info_list] # 主要信息
    
    today_data = pd.DataFrame([i['today'] for i in data ]) # 生成today的数据
    today_data.columns = ['today_'+i for i in today_data.columns] # 修改列名
    
    total_data = pd.DataFrame([i['total'] for i in data ]) # 生成total的数据
    total_data.columns = ['total_'+i for i in total_data.columns] # 修改列名
    
    return pd.concat([info,total_data,today_data],axis=1) # info、today和total横向合并最终得到汇总的数据

today_province = get_data(data_province,['id','lastUpdateTime','name'])
today_province.head()

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_newConfirm,total_newDead,total_newHeal,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm
0,420000,2021-01-07 08:22:59,湖北,68149,0,63637,4512,0,0,NaN,NaN,NaN,0,None,0,0,None,0
1,810000,2021-01-07 17:13:41,香港,9107,0,8258,154,0,0,NaN,NaN,NaN,25,None,131,1,None,-74
2,440000,2021-01-07 08:37:57,广东,2062,0,2018,8,0,0,NaN,NaN,NaN,2,None,3,0,None,-1
3,310000,2021-01-07 15:48:25,上海,1543,0,1442,7,0,0,NaN,NaN,NaN,5,None,5,0,None,0
4,330000,2021-01-07 08:22:58,浙江,1306,0,1293,1,0,0,NaN,NaN,NaN,0,None,0,0,None,0


In [16]:
def save_data(data,name): # 定义保存数据方法
    file_name = name+'_'+time.strftime('%Y_%m_%d',time.localtime(time.time()))+'.csv'
    data.to_csv(file_name,index=None,encoding='utf_8_sig')
    print(file_name+' 保存成功！')

time.strftime('%Y_%m_%d',time.localtime(time.time()))

'2021_01_07'

In [17]:
save_data(today_province,'today_province')

today_province_2021_01_07.csv 保存成功！


# 世界各国实时数据爬取

In [18]:
areaTree = data['areaTree'] # 取出areaTree

areaTree[0] # 查看第一个国家的数据

{'today': {'confirm': 2820,
  'suspect': None,
  'heal': 1275,
  'dead': 70,
  'severe': None,
  'storeConfirm': None},
 'total': {'confirm': 149881,
  'suspect': 0,
  'heal': 111599,
  'dead': 5004,
  'severe': 0,
  'input': 0},
 'extData': {},
 'name': '突尼斯',
 'id': '9577772',
 'lastUpdateTime': '2021-01-07 08:23:02',
 'children': []}

In [19]:
for i in range(len(areaTree)):  # 查看各国家名称和更新时间
    print(areaTree[i]['name'],areaTree[i]['lastUpdateTime'])
    if i == 5:
        break

突尼斯 2021-01-07 08:23:02
塞尔维亚 2021-01-07 08:23:02
中国 2021-01-07 17:13:40
日本本土 2021-01-07 18:03:37
泰国 2021-01-07 14:23:37
新加坡 2021-01-07 18:03:38


In [20]:
#从areaTree中提取每个国家的实时数据
today_world = get_data(areaTree,['id','lastUpdateTime','name'])
today_world.head()

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input
0,9577772,2021-01-07 08:23:02,突尼斯,149881,0,111599,5004,0,0.0,2820.0,NaN,1275.0,70.0,NaN,NaN,NaN
1,9507896,2021-01-07 08:23:02,塞尔维亚,352120,0,0,3444,0,0.0,2882.0,NaN,0.0,0.0,NaN,NaN,NaN
2,0,2021-01-07 17:13:40,中国,97254,0,91172,4795,0,4359.0,94.0,NaN,164.0,1.0,NaN,-38.0,11.0
3,1,2021-01-07 18:03:37,日本本土,266098,0,210295,3636,0,0.0,6603.0,NaN,3412.0,27.0,NaN,NaN,NaN
4,2,2021-01-07 14:23:37,泰国,9636,0,4521,67,0,0.0,305.0,NaN,103.0,1.0,NaN,NaN,NaN


In [21]:
save_data(today_world,'today_world')

today_world_2021_01_07.csv 保存成功！


# 历史数据处理

# 全国历史数据爬取


In [22]:
data.keys()

dict_keys(['chinaTotal', 'chinaDayList', 'lastUpdateTime', 'overseaLastUpdateTime', 'areaTree'])

In [23]:
chinaDayList = data['chinaDayList'] # 取出chinaDayList

In [24]:
type(chinaDayList) # 查看chinaDayList的格式

list

In [25]:
chinaDayList[0]

{'date': '2020-01-13',
 'today': {'confirm': 0,
  'suspect': 0,
  'heal': 0,
  'dead': 0,
  'severe': None,
  'storeConfirm': 0,
  'input': 0},
 'total': {'confirm': 41,
  'suspect': 0,
  'heal': 0,
  'dead': 1,
  'severe': None,
  'input': 0,
  'storeConfirm': 0},
 'extData': None,
 'lastUpdateTime': None}

In [26]:
alltime_China = get_data(chinaDayList,['date','lastUpdateTime'])
alltime_China.head()

,date,lastUpdateTime,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_storeConfirm,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input
0,2020-01-13,None,41,0,0,1,None,0,0,0,0,0,0,None,0,0
1,2020-01-14,None,41,0,0,1,None,0,0,0,0,0,0,None,0,0
2,2020-01-15,None,41,0,5,2,None,0,0,0,0,0,0,None,0,0
3,2020-01-16,None,45,0,8,2,None,0,0,0,0,0,0,None,0,0
4,2020-01-17,None,62,0,12,2,None,0,0,0,0,0,0,None,0,0


In [27]:
save_data(alltime_China,'alltime_China')

alltime_China_2021_01_07.csv 保存成功！


# 全国各省历史数据爬取

In [28]:
#以其中一个省为例，先尝试获取其历史数据，其他的省可以使用同样的方法。
url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode=420000' # 定义数据地址
r = requests.get(url, headers=headers) # 进行请求
data_json = json.loads(r.text) # 获取json数据

data_json.keys()

dict_keys(['reqId', 'code', 'msg', 'data', 'timestamp'])

In [29]:
#在list中存放着数据。
data_json['data']['list'][0]

{'date': '2020-01-20',
 'today': {'confirm': 0,
  'suspect': 0,
  'heal': 0,
  'dead': 0,
  'severe': None,
  'storeConfirm': 0,
  'input': 0},
 'total': {'confirm': 270,
  'suspect': 0,
  'heal': 25,
  'dead': 6,
  'severe': None,
  'input': 0,
  'storeConfirm': 239},
 'extData': None,
 'lastUpdateTime': None}

In [30]:
data_test = get_data(data_json['data']['list'],['date'])
data_test['name'] = '湖北省'
data_test.head()

,date,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_storeConfirm,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input,name
0,2020-01-20,270,0,25,6,None,0,239,0,0,0,0,None,0,0,湖北省
1,2020-01-21,375,0,28,9,None,0,338,105,0,3,3,None,0,0,湖北省
2,2020-01-22,444,0,28,17,None,0,399,69,0,0,8,None,0,0,湖北省
3,2020-01-23,549,0,31,24,None,0,494,105,0,3,7,None,0,0,湖北省
4,2020-01-24,729,0,32,39,None,0,658,180,0,1,15,None,0,0,湖北省


In [31]:
today_province[['id','name']].head()

,id,name
0,420000,湖北
1,810000,香港
2,440000,广东
3,310000,上海
4,330000,浙江


In [32]:
#为了便于写入各省的名称，我们需要生成一个各省行政代码和省名称对应的字典。
a = ['1','2','3','4']
b = ['q','w','e','r']

for i,j in zip(a, b):
    print(i,j)

1 q
2 w
3 e
4 r


In [33]:
{ i:j  for i,j in zip(a, b)}

{'1': 'q', '2': 'w', '3': 'e', '4': 'r'}

In [34]:
province_dict = {num:name for num,name in zip(today_province['id'],today_province['name'])}

# 查看前五个内容
count = 0
for i in province_dict:
    print(i,province_dict[i])
    count += 1
    if count == 5:
        break

420000 湖北
810000 香港
440000 广东
310000 上海
330000 浙江


In [35]:
#每一个省的列名是相同的，因此多个省的数据合并起来就可以存入一个数据中，数据合并演示的例子如下：
df1 = pd.DataFrame([{'a':1,'b':2,'c':3,},{'a':111,'b':222}])
df1

,a,b,c
0,1,2,3.0
1,111,222,NaN


In [36]:
df2 = pd.DataFrame([{'a':9,'b':8,'c':7,},{'a':345,'c':789}])
df2

,a,b,c
0,9,8.0,7
1,345,NaN,789


In [37]:
df1 = pd.concat([df1,df2],axis=0)
df1

,a,b,c
0,1,2.0,3.0
1,111,222.0,NaN
0,9,8.0,7.0
1,345,NaN,789.0


In [38]:
#按照上述方法，得到每一个省的数据后，进行合并。
start = time.time()
for province_id in province_dict: # 遍历各省编号
    
    try:
        # 按照省编号访问每个省的数据地址，并获取json数据
        url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode='+province_id
        r = requests.get(url, headers=headers)
        data_json = json.loads(r.text)
        
        # 提取各省数据，然后写入各省名称
        province_data = get_data(data_json['data']['list'],['date'])
        province_data['name'] = province_dict[province_id]
        
        # 合并数据
        if province_id == '420000':
            alltime_province = province_data
        else:
            alltime_province = pd.concat([alltime_province,province_data])
            
        print('-'*20,province_dict[province_id],'成功',
              province_data.shape,alltime_province.shape,
              ',累计耗时:',round(time.time()-start),'-'*20)
        
        # 设置延迟等待
        time.sleep(10)
        
    except:
        print('-'*20,province_dict[province_id],'wrong','-'*20)

-------------------- 湖北 成功 (354, 16) (354, 16) ,累计耗时: 0 --------------------
-------------------- 香港 成功 (354, 16) (708, 16) ,累计耗时: 11 --------------------
-------------------- 广东 成功 (352, 16) (1060, 16) ,累计耗时: 21 --------------------
-------------------- 上海 成功 (338, 16) (1398, 16) ,累计耗时: 31 --------------------
-------------------- 浙江 成功 (350, 16) (1748, 16) ,累计耗时: 42 --------------------
-------------------- 河南 成功 (352, 16) (2100, 16) ,累计耗时: 52 --------------------
-------------------- 湖南 成功 (349, 16) (2449, 16) ,累计耗时: 62 --------------------
-------------------- 北京 成功 (346, 16) (2795, 16) ,累计耗时: 73 --------------------
-------------------- 安徽 成功 (351, 16) (3146, 16) ,累计耗时: 83 --------------------
-------------------- 新疆 成功 (354, 16) (3500, 16) ,累计耗时: 93 --------------------
-------------------- 黑龙江 成功 (343, 16) (3843, 16) ,累计耗时: 104 --------------------
-------------------- 江西 成功 (352, 16) (4195, 16) ,累计耗时: 114 --------------------
-------------------- 山东 成功 (346, 16) (4541, 16) ,累计耗

In [39]:
save_data(alltime_province,'alltime_province')

alltime_province_2021_01_07.csv 保存成功！


# 世界各国历史数据爬取

In [40]:
#首先尝试爬取意大利的历史数据
url_italy = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode=15'  # 意大利的数据地址
r = requests.get(url_italy, headers=headers) # 进行访问
italy_json = json.loads(r.text) # 导出json数据

italy_json.keys()

dict_keys(['reqId', 'code', 'msg', 'data', 'timestamp'])

In [41]:
italy_json['data'].keys()

dict_keys(['list'])

In [42]:
italy_json['data']['list'][0] # 查看数据内容

{'date': '2020-01-31',
 'today': {'confirm': 2,
  'suspect': 0,
  'heal': 0,
  'dead': 0,
  'severe': None,
  'storeConfirm': None},
 'total': {'confirm': 2,
  'suspect': 0,
  'heal': 0,
  'dead': 0,
  'severe': 0,
  'input': 0},
 'extData': None,
 'lastUpdateTime': None}

In [43]:
#每个国家的数据格式和之前各省的几乎一样，因此使用定义好的方法生成数据，然后把国家名称写入数据。
data_italy = get_data(italy_json['data']['list'],['date']) # 生成数据
data_italy['name'] = '意大利' # 写入国家名称
data_italy.head()

,date,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,today_input,name
0,2020-01-31,2,0,0,0,0,0,2,0.0,0,0,NaN,None,NaN,意大利
1,2020-02-01,2,0,0,0,0,0,0,0.0,0,0,NaN,None,0.0,意大利
2,2020-02-02,2,0,0,0,0,0,0,0.0,0,0,NaN,None,0.0,意大利
3,2020-02-03,2,0,0,0,0,0,0,0.0,0,0,NaN,None,0.0,意大利
4,2020-02-04,2,0,0,0,0,0,0,0.0,0,0,NaN,None,0.0,意大利


In [44]:
#因为原始数据中没有国家名称，为了得到每个国家的名称，需要生成国家编号和国家名称的键值对，
#这样就可以存储国家名称，在之前的世界各国实时数据today_world中有国家的编号和名称，可以用它来生成键值对。

today_world[['id','name']].head()
country_dict = {key:value for key,value in zip(today_world['id'], today_world['name'])}

# 查看前五个内容
count = 0
for i in country_dict:
    print(i,country_dict[i])
    count += 1
    if count == 5:
        break

9577772 突尼斯
9507896 塞尔维亚
0 中国
1 日本本土
2 泰国


In [45]:
#通过每个国家的编号访问每个国家历史数据的地址，然后获取每一个国家的历史数据
start = time.time()
for country_id in country_dict: # 遍历每个国家的编号
    
    try:
        # 按照编号访问每个国家的数据地址，并获取json数据
        url = 'https://c.m.163.com/ug/api/wuhan/app/data/list-by-area-code?areaCode='+country_id
        r = requests.get(url, headers=headers)
        json_data = json.loads(r.text)
        
        # 生成每个国家的数据
        country_data = get_data(json_data['data']['list'],['date'])
        country_data['name'] = country_dict[country_id]

        # 数据叠加
        if country_id == '9577772':
            alltime_world = country_data
        else:
            alltime_world = pd.concat([alltime_world,country_data])
            
        print('-'*20,country_dict[country_id],'成功',country_data.shape,alltime_world.shape,
              ',累计耗时:',round(time.time()-start),'-'*20)
        
        time.sleep(10)

    except:
        print('-'*20,country_dict[country_id],'wrong','-'*20)


-------------------- 突尼斯 成功 (293, 15) (293, 15) ,累计耗时: 0 --------------------
-------------------- 塞尔维亚 成功 (298, 15) (591, 15) ,累计耗时: 11 --------------------
-------------------- 中国 成功 (335, 15) (926, 15) ,累计耗时: 21 --------------------
-------------------- 日本本土 成功 (312, 15) (1238, 15) ,累计耗时: 31 --------------------
-------------------- 泰国 成功 (309, 15) (1547, 15) ,累计耗时: 42 --------------------
-------------------- 新加坡 成功 (326, 15) (1873, 15) ,累计耗时: 52 --------------------
-------------------- 韩国 成功 (349, 15) (2222, 15) ,累计耗时: 63 --------------------
-------------------- 澳大利亚 成功 (308, 15) (2530, 15) ,累计耗时: 73 --------------------
-------------------- 德国 成功 (315, 15) (2845, 15) ,累计耗时: 83 --------------------
-------------------- 美国 成功 (320, 15) (3165, 15) ,累计耗时: 94 --------------------
-------------------- 马来西亚 成功 (312, 15) (3477, 15) ,累计耗时: 104 --------------------
-------------------- 越南 成功 (283, 15) (3760, 15) ,累计耗时: 114 --------------------
-------------------- 圣巴泰勒米 成功 (30, 15) (3790

-------------------- 阿根廷 成功 (299, 15) (23773, 15) ,累计耗时: 1048 --------------------
-------------------- 法属波利尼西亚 成功 (36, 15) (23809, 15) ,累计耗时: 1058 --------------------
-------------------- 巴林 成功 (304, 15) (24113, 15) ,累计耗时: 1069 --------------------
-------------------- 莫桑比克 成功 (266, 15) (24379, 15) ,累计耗时: 1079 --------------------
-------------------- 喀麦隆 成功 (280, 15) (24659, 15) ,累计耗时: 1089 --------------------
-------------------- 乌干达 成功 (277, 15) (24936, 15) ,累计耗时: 1100 --------------------
-------------------- 厄立特里亚 成功 (50, 15) (24986, 15) ,累计耗时: 1110 --------------------
-------------------- 刚果（布） 成功 (261, 15) (25247, 15) ,累计耗时: 1120 --------------------
-------------------- 津巴布韦 成功 (265, 15) (25512, 15) ,累计耗时: 1131 --------------------
-------------------- 丹麦 成功 (309, 15) (25821, 15) ,累计耗时: 1141 --------------------
-------------------- 阿鲁巴 成功 (34, 15) (25855, 15) ,累计耗时: 1151 --------------------
-------------------- 斐济 成功 (244, 15) (26099, 15) ,累计耗时: 1162 --------------------


-------------------- 安道尔 成功 (278, 15) (45262, 15) ,累计耗时: 2076 --------------------
-------------------- 牙买加 成功 (262, 15) (45524, 15) ,累计耗时: 2086 --------------------
-------------------- 格恩西岛 成功 (29, 15) (45553, 15) ,累计耗时: 2097 --------------------
-------------------- 罗马尼亚 成功 (302, 15) (45855, 15) ,累计耗时: 2107 --------------------
-------------------- 阿曼 成功 (303, 15) (46158, 15) ,累计耗时: 2117 --------------------
-------------------- 列支敦士登 成功 (48, 15) (46206, 15) ,累计耗时: 2128 --------------------
-------------------- 马达加斯加 成功 (266, 15) (46472, 15) ,累计耗时: 2138 --------------------


In [46]:
save_data(alltime_world,'alltime_world')

alltime_world_2021_01_07.csv 保存成功！
